# Intro
This notebook is used to generate a labeled logging scars dataset. The data is hand labeled by our team using geojson.io.

## Installing packages

In [1]:
!pip install -q geemap
!pip install -q retry
!pip install -q rasterio
!pip install -q shapely
!pip install -q geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 41.4 MB/s eta 0:00:00


## Importing Libraries

In [2]:
import ee
import rasterio
import numpy as np
from rasterio.windows import Window
import time
import pandas as pd
import matplotlib.pyplot as plt
from osgeo import gdal
import os
from os.path import isfile, join
import geopandas as gpd
from rasterio.features import geometry_mask
import os
import pyarrow as pa
import pyarrow.parquet as pq
import json
from shapely.geometry import shape, MultiPolygon
from geopy import Point
from geopy.distance import geodesic

from google.colab import drive

## Setup GEE and Drive
You need to connect to GEE to download sentinel-2 satellite images and to Google Drive to retrieve those satellite images and other files.

In [7]:
drive.mount('/content/drive')

# Authenticate to the Earth Engine servers
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Download Geojsons Data
Download the geojsons data which is a label produced by our team. You will need to change the directory to reflect your working directory.

In [8]:
!cp -nr /content/drive/MyDrive/GreenGuardians/data/geojsons .

# Download Image
We download the satellite data for each location with at least a given diagonal distance. It will take some time to download all images. You can optionally turn off the export part if you already have the data and avoid the mistake of re-running it.

In [9]:
# You can disable exporting to avoid re-generating satellite images again and again
disable_export = False
distance = 400  # 362.04 meters for 256 px
final_patch_size = 256
image_prefix = 'scars'

# List of locations to download satellite image
locations = {
    "caribou-lake-710_1": {
        "Longitude": -89.07180431518495,
        "Latitude": 50.26474654637079,
        "bearing": 315
    },
    "caribou-lake-710_2": {
        "Longitude": -89.0922648771783,
        "Latitude": 50.2648353575554,
        "bearing": 315
    },
    "caribou-lake-708_1": {
        "Longitude": -89.11212491207844,
        "Latitude": 50.27654183145796,
        "bearing": 315
    },
    "caribou-lake-701_1": {
        "Longitude": -89.2148123819986,
        "Latitude": 50.459890567819855,
        "bearing": 315
    },
    "caribou-lake-702_1": {
        "Longitude": -89.2212951993255,
        "Latitude": 50.45073363010053,
        "bearing": 315
    },
    "gibson-502_1": {
        "Longitude": -89.91218052883151,
        "Latitude": 50.10632662287529,
        "bearing": 225
    },
    "hillside360_1": {
        "Longitude": -91.01160817534075,
        "Latitude": 50.383020589810656,
        "bearing": 315
    },
    "hillside360_2": {
        "Longitude": -91.05272790875333,
        "Latitude": 50.38292259524923,
        "bearing": 45
    },
    "hillside360_3": {
        "Longitude": -91.01160283511261,
        "Latitude": 50.422466149813204,
        "bearing": 225
    },
    "hillside360_4": {
        "Longitude": -91.0244850289575,
        "Latitude": 50.44797663658497,
        "bearing": 135
    },
    "hwy6424XX_1": {
        "Longitude": -91.49548037262977,
        "Latitude": 49.88899955070093,
        "bearing": 45
    },
    "hwy6424XX_2": {
        "Longitude": -91.67051135755811,
        "Latitude": 49.9154086831214,
        "bearing": 45
    },
    "hwy6424XX_3": {
        "Longitude": -91.63123211433263,
        "Latitude": 49.940773818003834,
        "bearing": 225
    },
    "keershaw-503_1": {
        "Longitude": -90.00421792483466,
        "Latitude": 49.85188270608046,
        "bearing": 45
    },
    "keershaw-504_1": {
        "Longitude": -89.86915213699602,
        "Latitude": 49.89758643086071,
        "bearing": 225
    },
    "kiwi201_1": {
        "Longitude": -91.18163791726313,
        "Latitude": 50.554278569030856,
        "bearing": 225
    },
    "lacseul010": {
        "Longitude": -91.51614531159406,
        "Latitude": 50.631587126749025,
        "bearing": 225
    },
    "obonga-605": {
        "Longitude": -89.47640907598071,
        "Latitude": 50.12663668435772,
        "bearing": 45
    },
    "trist101": {
        "Longitude": -91.10001522228801,
        "Latitude": 50.86940938780839,
        "bearing": 45
    },
    "trist114_1": {
        "Longitude": -91.20102663691645,
        "Latitude": 50.877626598945994,
        "bearing": 45
    },
    "trist114_2": {
        "Longitude": -91.16316907612344,
        "Latitude": 50.87762238010532,
        "bearing": 315
    },
    "trist130_1": {
        "Longitude": -91.2411569764188,
        "Latitude": 50.90624337484397,
        "bearing": 315
    },
    "watin401_1": {
        "Longitude": -91.2345854074319,
        "Latitude": 50.27233819833512,
        "bearing": 45
    },
    "watin402_1": {
        "Longitude": -91.23122298152398,
        "Latitude": 50.296070473364665,
        "bearing": 45
    },
    "Whitesand-805a_1": {
        "Longitude": -88.67804883458872,
        "Latitude": 50.514464542040514,
        "bearing": 45
    },
    "Whitesand-804_1": {
        "Longitude": -88.75072886594884,
        "Latitude": 50.53906802149277,
        "bearing": 225
    },
    "whitesand-805b_1": {
        "Longitude": -88.64642393435341,
        "Latitude": 50.52669782402512,
        "bearing": 135
    }
}

loc_boxes = []
# Building a list of rectangular shapes to be used as a region to download satellite images
for name in locations:
    start = Point(locations[name]["Latitude"], locations[name]["Longitude"])
    bearing = locations[name]["bearing"]

    # Calculate destination point
    destination = geodesic(meters=distance*10).destination(point=start, bearing=bearing)

    # Define points
    point1 = ee.Geometry.Point([destination.longitude, destination.latitude])
    point2 = ee.Geometry.Point([locations[name]["Longitude"], locations[name]["Latitude"]])

    point1_coords = point1.coordinates()
    point2_coords = point2.coordinates()

    llx = min(point1_coords.get(0).getInfo(), point2_coords.get(0).getInfo())
    lly = min(point1_coords.get(1).getInfo(), point2_coords.get(1).getInfo())
    urx = max(point1_coords.get(0).getInfo(), point2_coords.get(0).getInfo())
    ury = max(point1_coords.get(1).getInfo(), point2_coords.get(1).getInfo())

    # Create a rectangle using the min and max points
    rect = ee.Geometry.Rectangle([llx, lly, urx, ury])
    loc_boxes.append([name, rect, bearing])

print(f"There are {len(loc_boxes)} locations.")

There are 27 locations.


In [ ]:
# Removing clouds
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask)

# Set image resolution
resolution = 10  # Sentinel-2 has a resolution of 10m for RGB bands

# You can disable exporting to avoid re-generating satellite images again and again
if not disable_export:
    for rect in loc_boxes:
        print(f"Processing {rect[0]} location.")

        # Harmonized Sentinel-2 MSI: MultiSpectral Instrument, Level-2A
        image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                          .filterDate('2023-05-01', '2023-09-30') \
                          .filterBounds(rect[1]) \
                          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                          .map(maskS2clouds).select('B4', 'B3', 'B2', 'B8').median()

        # Clip the image to the rectangle
        image_rect = image.clip(rect[1])

        # Define the export parameters
        task_config = {
            'image': image_rect,
            'folder': f'Tmp_GEE_Boreal_V1_{final_patch_size}',
            'description': f'{image_prefix}_{rect[0]}',
            'scale': resolution,
            'region': rect[1].getInfo()['coordinates'],
            'maxPixels': 1e13
        }

        # Export the image to Google Drive
        task = ee.batch.Export.image.toDrive(**task_config)
        task.start()

        # Monitor the task
        while task.status()['state'] in ['READY', 'RUNNING']:
            print(task.status())
            time.sleep(15)
        else:
            print(task.status())

Processing caribou-lake-710_1 location.
{'state': 'READY', 'description': 'scars_caribou-lake-710_1', 'creation_timestamp_ms': 1688074355280, 'update_timestamp_ms': 1688074355280, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'TXBT6W5SNJQ2IB6BO2MKK4E7', 'name': 'projects/earthengine-legacy/operations/TXBT6W5SNJQ2IB6BO2MKK4E7'}
{'state': 'READY', 'description': 'scars_caribou-lake-710_1', 'creation_timestamp_ms': 1688074355280, 'update_timestamp_ms': 1688074355280, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'TXBT6W5SNJQ2IB6BO2MKK4E7', 'name': 'projects/earthengine-legacy/operations/TXBT6W5SNJQ2IB6BO2MKK4E7'}
{'state': 'RUNNING', 'description': 'scars_caribou-lake-710_1', 'creation_timestamp_ms': 1688074355280, 'update_timestamp_ms': 1688074373944, 'start_timestamp_ms': 1688074373869, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'TXBT6W5SNJQ2IB6BO2MKK4E7', 'name': 'projects/earthengine-legacy/operations/TXBT6W5SNJQ2IB6BO2MKK4E7'}
{'state': 'RUNNING', 'd

# Generate Dataset
After converting it to a binary mask, we put together the satellite image and the labeled data in geojson format.

In [ ]:
geojson_dir = '/content/geojsons/'
patch_remote_dir = f"/content/drive/MyDrive/Tmp_GEE_Boreal_V1_{final_patch_size}/"
patch_dir = f"/content/Tmp_GEE_Boreal_V1_{final_patch_size}/"
dataset_dir = "/content/dataset/"
dataset_image_dir = f"{dataset_dir}image/"
dataset_mask_dir = f"{dataset_dir}mask/"

!rm -rf {patch_dir}
!rm -rf {dataset_dir}

!cp -rn {patch_remote_dir} .

!mkdir -p {dataset_dir}
!mkdir -p {dataset_image_dir}
!mkdir -p {dataset_mask_dir}

# Reading the GeoJSON file for masks
data_files, polygons = [], []
for subdir, dirs, files in os.walk(geojson_dir):
    for f in files:
        if isfile(join(subdir, f)) and '.geojson' in f:
            data_files.append(join(subdir, f))
            gdf = gpd.read_file(join(subdir, f))
            for idx, row in gdf.iterrows():
                polygons.append(row.geometry)

# Generating a mask for each patch
for loc_box in loc_boxes:
    file_path = f'{patch_dir}{image_prefix}_{loc_box[0]}.tif'

    # Load GeoTIFF file
    with rasterio.open(file_path) as src:
        if loc_box[2] == 315:
            window = Window(src.width-final_patch_size, src.height-final_patch_size, final_patch_size, final_patch_size) # Takes bottom right
        elif loc_box[2] == 225:
            window = Window(src.width-final_patch_size, 0, final_patch_size, final_patch_size) # Takes top right
        elif loc_box[2] == 45:
            window = Window(0, src.height-final_patch_size, final_patch_size, final_patch_size) # Takes top right
        elif loc_box[2] == 135:
            window = Window(0, 0, final_patch_size, final_patch_size) # Takes top right
        else:
            print("Bearing is not recognized!")
        subset = src.read(window=window)
        kwargs = src.meta.copy()
        kwargs.update({
            'height': window.height,
            'width': window.width,
            'transform': src.window_transform(window)
        })

        # Saving the resized version of image
        file_name = os.path.basename(file_path)
        with rasterio.open(f"{dataset_image_dir}{file_name}", "w", **kwargs) as dst:
            dst.write(subset)

        # Create the mask
        mask = geometry_mask(polygons, transform=dst.transform, out_shape=(window.height, window.width), all_touched = True, invert=True)

        # Prepare the metadata for the mask file.
        mask_meta = dst.meta.copy()
        mask_meta.update({
            'dtype': 'uint8',
            'count': 1,
        })

        # Save the mask to a GeoTIFF file.
        with rasterio.open(f"{dataset_mask_dir}{file_name}", 'w', **mask_meta) as mask_dst:
            mask_dst.write(mask.astype(rasterio.uint8), 1)

# Dataset View
A demonstration of what the generated dataset looks like.

In [ ]:
def loadTiff(in_image, max=-1):
    src = gdal.Open(in_image)
    nbands = src.RasterCount
    in_band = src.GetRasterBand(1)
    xinit, yinit = (0, 0)
    block_xsize, block_ysize = (in_band.XSize, in_band.YSize)
    # read the (multiband) file into an array
    image = src.ReadAsArray(xinit, yinit, block_xsize, block_ysize)
    if max != -1:
        for i in range(nbands):
          image[i] = np.clip(image[i]/max, 0, 1)
        # reshape from bandsxheightxwidth to wxhxb
        image = np.moveaxis(image, 0, -1)
    return image, block_ysize, block_xsize, nbands

In [ ]:
# Reading the patches
patch_image_dir = f'/content/dataset/image/'
patch_mask_dir = f'/content/dataset/mask/'

tiles = []
for subdir, dirs, files in os.walk(patch_image_dir):
    for f in files:
        if isfile(join(patch_image_dir, f)) and '.tif' in f:
            tiles.append([join(patch_image_dir, f), join(patch_mask_dir, f)])

count = 0
for t in tiles:
    with rasterio.open(t[1]) as src:
        mask = src.read(1)

    fig, ax = plt.subplots(1, 2, figsize=(16, 15))
    [img, xsize, ysize, nbands] = loadTiff(t[0], 2000)
    img_rgb = np.dstack((img[:,:,0], img[:,:,1], img[:,:,2]))
    # Create a red color overlay using the mask
    overlay = np.zeros((img_rgb.shape[0], img_rgb.shape[1], 4))  # 4 for RGBA
    overlay[..., 0] = 1  # Red channel
    overlay[..., 3] = mask

    mask_file_name = os.path.basename(t[0]).split('_')[1]

    # Display the original image and the masked image
    ax[0].imshow(img_rgb)
    ax[0].set_title(f'Original Patch - {mask_file_name}')
    ax[0].axis('off')

    ax[1].imshow(img_rgb, cmap='gray')
    ax[1].imshow(overlay)
    ax[1].set_title(f'Labeled Patch - Mask sum: {mask.sum():,}')
    ax[1].axis('off')

    plt.show()

## Upload Dataset
Moving the generated dataset to a Google Drive for next steps like EDA and model building.

In [15]:
# Change the destination directory name
remote_destination_dir = f'/content/drive/MyDrive/GreenGuardians/data/dataset/logging_scars_V1_{final_patch_size}'
!mkdir -p {remote_destination_dir}
!cp -rn {dataset_dir}/* {remote_destination_dir}